# 🚀 Parallel BFS & Afforest Implementation in Julia

**Project 3: Parallel Processing - AUTH**

This notebook implements:
- **BFS (Breadth-First Search)** - GPU-accelerated graph traversal
- **Afforest Algorithm** - Union-Find with sampling for connected components
- **MAT File Loading** - Load MATLAB sparse matrices (Friendster, Mawi)

---

## 1. Install Julia Kernel (Run First!)

After running these cells, **restart runtime** and change kernel to Julia.

In [ ]:
%%shell
set -e
JULIA_VERSION="1.10.2"

if [ ! -d "/usr/local/julia" ]; then
    echo "📥 Installing Julia $JULIA_VERSION..."
    wget -q https://julialang-s3.julialang.org/bin/linux/x64/1.10/julia-${JULIA_VERSION}-linux-x86_64.tar.gz
    tar -xzf julia-${JULIA_VERSION}-linux-x86_64.tar.gz
    mv julia-${JULIA_VERSION} /usr/local/julia
    rm julia-${JULIA_VERSION}-linux-x86_64.tar.gz
    ln -sf /usr/local/julia/bin/julia /usr/local/bin/julia
fi
julia --version

In [ ]:
%%shell
julia -e 'using Pkg; Pkg.add("IJulia"); using IJulia; installkernel("Julia")'
echo "✅ Restart runtime and select Julia kernel!"

---
## ⚠️ RESTART RUNTIME → Change Kernel to Julia

---

## 2. Install Packages

In [ ]:
using Pkg
Pkg.add(["CUDA", "Graphs", "SparseArrays", "BenchmarkTools", "HDF5", "MAT"])
println("✅ Packages installed!")

In [ ]:
using CUDA, Graphs, SparseArrays, BenchmarkTools, Statistics, Random, HDF5
println("✅ Packages loaded!")
println("CUDA available: ", CUDA.functional())
CUDA.functional() && println("GPU: ", CUDA.name(CUDA.device()))

---
## 3. Graph Data Structures

In [ ]:
const UNVISITED = typemax(Int32)

struct CSRGraph
    num_nodes::Int32
    num_edges::Int64
    row_ptr::Vector{Int64}
    col_idx::Vector{Int32}
end

function print_graph_stats(g::CSRGraph)
    degrees = [g.row_ptr[i+1] - g.row_ptr[i] for i in 1:g.num_nodes]
    println("📊 Nodes: $(g.num_nodes), Edges: $(g.num_edges)")
    println("   Degrees: min=$(minimum(degrees)), max=$(maximum(degrees)), avg=$(round(mean(degrees), digits=1))")
end

println("✅ CSRGraph defined!")

---
## 4. Graph Loading Functions

Supports:
- **MAT files** (MATLAB HDF5 format - Friendster, Mawi)
- **Text files** (edge list format)
- **CSRBin files** (binary CSR format)

In [ ]:
"""
Load graph from MATLAB .mat file (HDF5 format).
Structure: /Problem/A/ir (col_idx), /Problem/A/jc (row_ptr)
"""
function load_graph_mat(filename::String)
    println("📥 Loading MAT file: $filename")
    
    h5open(filename, "r") do file
        A = file["Problem"]["A"]
        
        println("  Reading jc (row pointers)...")
        jc = read(A["jc"])
        
        println("  Reading ir (column indices)...")
        ir = read(A["ir"])
        
        # Convert 0-indexed to 1-indexed
        row_ptr = Vector{Int64}(jc) .+ 1
        col_idx = Vector{Int32}(ir) .+ 1
        
        num_nodes = Int32(length(row_ptr) - 1)
        num_edges = Int64(length(col_idx))
        
        println("  ✅ Loaded: $num_nodes nodes, $num_edges edges")
        return CSRGraph(num_nodes, num_edges, row_ptr, col_idx)
    end
end

"""
Load graph from text file (edge list format).
Format: First line = num_nodes num_edges, then src dst pairs
"""
function load_graph_text(filename::String)
    println("📥 Loading text file: $filename")
    
    lines = readlines(filename)
    header = split(lines[1])
    num_nodes = parse(Int, header[1])
    
    adj = [Int32[] for _ in 1:num_nodes]
    
    for line in lines[2:end]
        parts = split(line)
        length(parts) >= 2 || continue
        src = parse(Int32, parts[1]) + 1  # 0-indexed to 1-indexed
        dst = parse(Int32, parts[2]) + 1
        1 <= src <= num_nodes && 1 <= dst <= num_nodes && push!(adj[src], dst)
    end
    
    # Build CSR
    num_edges = sum(length.(adj))
    row_ptr = zeros(Int64, num_nodes + 1)
    col_idx = Vector{Int32}(undef, num_edges)
    
    idx = 1
    for i in 1:num_nodes
        row_ptr[i] = idx
        sort!(adj[i])
        for n in adj[i]
            col_idx[idx] = n
            idx += 1
        end
    end
    row_ptr[num_nodes + 1] = idx
    
    println("  ✅ Loaded: $num_nodes nodes, $num_edges edges")
    return CSRGraph(Int32(num_nodes), Int64(num_edges), row_ptr, col_idx)
end

"""
Load graph from binary CSR format (.csrbin).
"""
function load_graph_csrbin(filename::String)
    println("📥 Loading CSRBin: $filename")
    
    open(filename, "r") do f
        num_nodes = read(f, Int64)
        num_edges = read(f, Int64)
        
        row_ptr = Vector{Int64}(undef, num_nodes + 1)
        col_idx = Vector{Int32}(undef, num_edges)
        read!(f, row_ptr)
        read!(f, col_idx)
        
        # Convert to 1-indexed
        row_ptr .+= 1
        col_idx .+= 1
        
        println("  ✅ Loaded: $num_nodes nodes, $num_edges edges")
        return CSRGraph(Int32(num_nodes), Int64(num_edges), row_ptr, col_idx)
    end
end

"""
Generate random graph for testing.
"""
function generate_random_graph(num_nodes::Int, avg_degree::Int; seed=42)
    Random.seed!(seed)
    adj = [Int32[] for _ in 1:num_nodes]
    
    for i in 1:num_nodes
        for _ in 1:rand(1:2*avg_degree)
            n = rand(1:num_nodes)
            n != i && push!(adj[i], Int32(n))
        end
    end
    
    num_edges = sum(length.(adj))
    row_ptr = zeros(Int64, num_nodes + 1)
    col_idx = Vector{Int32}(undef, num_edges)
    
    idx = 1
    for i in 1:num_nodes
        row_ptr[i] = idx
        sort!(adj[i])
        for n in adj[i]
            col_idx[idx] = n
            idx += 1
        end
    end
    row_ptr[num_nodes + 1] = idx
    
    return CSRGraph(Int32(num_nodes), Int64(num_edges), row_ptr, col_idx)
end

println("✅ Graph loaders defined!")

---
## 5. BFS Implementation (CPU & GPU)

In [ ]:
# CPU BFS
function bfs_cpu(graph::CSRGraph, source::Int32)
    distances = fill(UNVISITED, graph.num_nodes)
    distances[source] = Int32(0)
    frontier = [source]
    level = Int32(0)
    
    while !isempty(frontier)
        next = Int32[]
        for node in frontier
            for i in graph.row_ptr[node]:(graph.row_ptr[node+1]-1)
                n = graph.col_idx[i]
                if distances[n] == UNVISITED
                    distances[n] = level + 1
                    push!(next, n)
                end
            end
        end
        frontier = next
        level += 1
    end
    return distances
end

println("✅ CPU BFS defined!")

In [ ]:
# GPU BFS Kernel
function bfs_kernel!(row_ptr, col_idx, distances, frontier, frontier_size,
                     next_frontier, next_frontier_size, current_level)
    tid = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    
    if tid <= frontier_size[]
        node = frontier[tid]
        for i in row_ptr[node]:(row_ptr[node+1]-1)
            neighbor = col_idx[i]
            old = CUDA.atomic_cas!(pointer(distances, neighbor), UNVISITED, current_level + Int32(1))
            if old == UNVISITED
                idx = CUDA.atomic_add!(pointer(next_frontier_size, 1), Int32(1)) + 1
                next_frontier[idx] = neighbor
            end
        end
    end
    return nothing
end

function bfs_gpu(graph::CSRGraph, source::Int32)
    d_row_ptr = CuArray(graph.row_ptr)
    d_col_idx = CuArray(graph.col_idx)
    d_distances = CUDA.fill(UNVISITED, graph.num_nodes)
    d_distances[source] = Int32(0)
    
    d_frontier = CuArray{Int32}(undef, graph.num_nodes)
    d_next = CuArray{Int32}(undef, graph.num_nodes)
    d_frontier[1] = source
    d_size = CuArray([Int32(1)])
    d_next_size = CuArray([Int32(0)])
    
    level = Int32(0)
    while true
        h_size = Array(d_size)[1]
        h_size == 0 && break
        d_next_size .= Int32(0)
        
        @cuda threads=256 blocks=cld(h_size, 256) bfs_kernel!(
            d_row_ptr, d_col_idx, d_distances, d_frontier, d_size,
            d_next, d_next_size, level)
        CUDA.synchronize()
        
        d_frontier, d_next = d_next, d_frontier
        d_size, d_next_size = d_next_size, d_size
        level += 1
    end
    return Array(d_distances)
end

CUDA.functional() ? println("✅ GPU BFS defined!") : println("⚠️ GPU BFS defined (no GPU)")

---
## 6. Afforest Algorithm (Connected Components)

Union-Find with random neighbor sampling for faster convergence.

In [ ]:
# CPU Afforest
function find_root!(parent, u)
    root = u
    while parent[root] != root
        root = parent[root]
    end
    # Path compression
    while parent[u] != root
        next_u = parent[u]
        parent[u] = root
        u = next_u
    end
    return root
end

function union!(parent, u, v)
    ru, rv = find_root!(parent, u), find_root!(parent, v)
    if ru != rv
        ru < rv ? (parent[rv] = ru) : (parent[ru] = rv)
        return true
    end
    return false
end

function afforest_cpu(graph::CSRGraph; sampling_rounds=2)
    n = graph.num_nodes
    parent = collect(Int32(1):Int32(n))
    
    println("🌲 Afforest: Sampling phase ($sampling_rounds rounds)")
    for r in 1:sampling_rounds
        Random.seed!(r + 42)
        for u in 1:n
            start_idx, end_idx = graph.row_ptr[u], graph.row_ptr[u+1] - 1
            deg = end_idx - start_idx + 1
            deg > 0 && union!(parent, Int32(u), graph.col_idx[start_idx + rand(0:deg-1)])
        end
        for i in 1:n; find_root!(parent, Int32(i)); end
    end
    
    println("🌲 Afforest: Hook phase")
    changed, iters = true, 0
    while changed
        changed = false
        iters += 1
        for u in 1:n
            for i in graph.row_ptr[u]:(graph.row_ptr[u+1]-1)
                union!(parent, Int32(u), graph.col_idx[i]) && (changed = true)
            end
        end
        for i in 1:n; find_root!(parent, Int32(i)); end
    end
    
    for i in 1:n; parent[i] = find_root!(parent, Int32(i)); end
    println("  ✅ Found $(length(unique(parent))) components in $iters iterations")
    return parent
end

println("✅ CPU Afforest defined!")

In [ ]:
# GPU Afforest Kernels
function init_parent_kernel!(parent, n)
    tid = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    tid <= n && (parent[tid] = Int32(tid))
    return nothing
end

function sample_kernel!(row_ptr, col_idx, parent, n, seed)
    tid = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    if tid <= n
        start_idx, end_idx = row_ptr[tid], row_ptr[tid+1] - 1
        deg = end_idx - start_idx + 1
        if deg > 0
            r = (seed * 1664525 + 1013904223 + tid) % typemax(UInt32)
            v = col_idx[start_idx + r % deg]
            pu, pv = parent[tid], parent[v]
            pv < pu && CUDA.atomic_min!(pointer(parent, pu), pv)
            pu < pv && CUDA.atomic_min!(pointer(parent, pv), pu)
        end
    end
    return nothing
end

function hook_kernel!(row_ptr, col_idx, parent, changed, n)
    tid = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    if tid <= n
        pu = parent[tid]
        for i in row_ptr[tid]:(row_ptr[tid+1]-1)
            pv = parent[col_idx[i]]
            if pv < pu
                old = CUDA.atomic_min!(pointer(parent, pu), pv)
                old > pv && (changed[] = Int32(1); pu = pv)
            end
        end
    end
    return nothing
end

function compress_kernel!(parent, n)
    tid = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    if tid <= n
        p = parent[tid]
        pp = parent[p]
        p != pp && (parent[tid] = pp)
    end
    return nothing
end

function afforest_gpu(graph::CSRGraph; sampling_rounds=2)
    n = Int32(graph.num_nodes)
    blocks = cld(n, 256)
    
    d_row_ptr = CuArray(graph.row_ptr)
    d_col_idx = CuArray(graph.col_idx)
    d_parent = CuArray{Int32}(undef, n)
    d_changed = CuArray([Int32(0)])
    
    @cuda threads=256 blocks=blocks init_parent_kernel!(d_parent, n)
    
    println("🌲 GPU Afforest: Sampling ($sampling_rounds rounds)")
    for r in 1:sampling_rounds
        @cuda threads=256 blocks=blocks sample_kernel!(d_row_ptr, d_col_idx, d_parent, n, UInt32(r+42))
        @cuda threads=256 blocks=blocks compress_kernel!(d_parent, n)
        CUDA.synchronize()
    end
    
    println("🌲 GPU Afforest: Hook phase")
    iters = 0
    while true
        d_changed .= Int32(0)
        iters += 1
        @cuda threads=256 blocks=blocks hook_kernel!(d_row_ptr, d_col_idx, d_parent, d_changed, n)
        @cuda threads=256 blocks=blocks compress_kernel!(d_parent, n)
        CUDA.synchronize()
        Array(d_changed)[1] == 0 && break
    end
    
    for _ in 1:5
        @cuda threads=256 blocks=blocks compress_kernel!(d_parent, n)
    end
    CUDA.synchronize()
    
    parent = Array(d_parent)
    println("  ✅ Found $(length(unique(parent))) components in $iters iterations")
    return parent
end

CUDA.functional() ? println("✅ GPU Afforest defined!") : println("⚠️ GPU Afforest defined (no GPU)")

---
## 7. Test with Random Graph

In [ ]:
# Generate test graph
graph = generate_random_graph(10000, 20)
print_graph_stats(graph)
source = Int32(1)

In [ ]:
# Test BFS
println("\n🔍 Testing BFS...")
@time cpu_dist = bfs_cpu(graph, source)
println("  CPU: Visited $(count(d -> d != UNVISITED, cpu_dist)) nodes, max distance = $(maximum(d for d in cpu_dist if d != UNVISITED))")

if CUDA.functional()
    bfs_gpu(graph, source)  # warmup
    @time gpu_dist = bfs_gpu(graph, source)
    println("  GPU: Visited $(count(d -> d != UNVISITED, gpu_dist)) nodes")
    println("  ✅ Results match: ", cpu_dist == gpu_dist)
end

In [ ]:
# Test Afforest
println("\n🔍 Testing Afforest (Connected Components)...")
@time cpu_cc = afforest_cpu(graph)

if CUDA.functional()
    afforest_gpu(graph)  # warmup
    @time gpu_cc = afforest_gpu(graph)
    # Component IDs may differ, but structure should match
    cpu_components = length(unique(cpu_cc))
    gpu_components = length(unique(gpu_cc))
    println("  ✅ Component counts match: ", cpu_components == gpu_components)
end

---
## 8. Benchmark

In [ ]:
using Printf

println("\n📈 Benchmark Results:")
println("="^60)

for n in [1000, 5000, 10000, 50000]
    g = generate_random_graph(n, 20)
    
    # BFS
    cpu_bfs = @elapsed bfs_cpu(g, Int32(1))
    
    # Afforest
    cpu_aff = @elapsed afforest_cpu(g)
    
    if CUDA.functional()
        bfs_gpu(g, Int32(1)); afforest_gpu(g)  # warmup
        gpu_bfs = @elapsed (bfs_gpu(g, Int32(1)); CUDA.synchronize())
        gpu_aff = @elapsed (afforest_gpu(g); CUDA.synchronize())
        
        @printf("N=%6d | BFS: CPU %7.2fms GPU %7.2fms (%.1fx) | Afforest: CPU %7.2fms GPU %7.2fms (%.1fx)\n",
                n, cpu_bfs*1000, gpu_bfs*1000, cpu_bfs/gpu_bfs,
                cpu_aff*1000, gpu_aff*1000, cpu_aff/gpu_aff)
    else
        @printf("N=%6d | BFS: CPU %7.2fms | Afforest: CPU %7.2fms\n", n, cpu_bfs*1000, cpu_aff*1000)
    end
end

---
## 9. Load Real Graphs (Optional)

Upload your MAT files to Colab, then load them:

In [ ]:
# Example: Load MAT file (upload first)
# graph = load_graph_mat("com-Friendster.mat")
# print_graph_stats(graph)

# Or load text file
# graph = load_graph_text("graph.txt")

# Or load CSRBin
# graph = load_graph_csrbin("graph.csrbin")

println("📁 Upload your graph files and uncomment the appropriate line above.")

---
## 📝 Summary

| Algorithm | Description | GPU Acceleration |
|-----------|-------------|------------------|
| **BFS** | Breadth-First Search | ✅ Parallel frontier expansion |
| **Afforest** | Union-Find with sampling | ✅ Atomic operations for merging |

### Graph Formats Supported:
- `.mat` - MATLAB HDF5 sparse matrix (Friendster, Mawi)
- `.txt` - Edge list text format
- `.csrbin` - Binary CSR format